In [1]:
#Imports
import json
import numpy as np
import pandas as pd
import csv
import os
from math import log
from pylab import lstsq
from pylab import matrix
from pylab import exp

In [2]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results,photo_name):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result_' + photo_name + '.json',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [3]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open(path,'r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [4]:
def slicesParser_max_peaksI(dictres):
#extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x) for x in range(0,len(allSlices[0]['max_peaks_intensities']))]    
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    for i in range(0,len(allSlices)):
        df_sum.loc[i] = allSlices[i]['max_peaks_intensities'][0:] 
    return df_sum

In [5]:
def slicesParser_halfpeakstime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']
    return df_sum

In [6]:
def slicesParser_amplitudes(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['amplitudes']

    return df_sum

In [7]:
def slicesParser_min_peaksI(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

    return df_sum

In [8]:
def slicesParser_peaktime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']

    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

    return df_sum

# Calcula el alternance ratio

AR_list = []
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
jsons_list = os.listdir(path)
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    slices = dic['slices']
    i = 0
    df = pd.DataFrame(columns=['transient1','transient2','transient_max'])
    print (file)
    for i in range(0,33):
        try:
            amps = slices[i]['amplitudes'][0:2]
            amps.append(max(amps))
            i += 1
            df.loc[len(df)] = amps
        except IndexError:
            pass
    df.to_csv(path + file + 'max_amp.csv', sep='\t')
print('Done')

In [12]:
# Guarda en tablas separadas los datos de cada json por parámetro

param_list = ['max_peaks_positions','min_peaks_positions','max_peaks_intensities','min_peaks_intensities','amplitudes','times_to_peaks','times_to_half_peaks']
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
path2 = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/'
for param in param_list:
    os.makedirs(path2 + param + '/')
    jsons_list = os.listdir(path)
    for file in jsons_list:
        path_file = path + file
        dic = jasonReader(path_file)
        slices = dic['slices']
        i = 0
        df = pd.DataFrame(columns=['transient1','transient2'])
        print (file)
        for i in range(0,33):
            try:
                amps = slices[i][param][0:2]
                i += 1
                df.loc[len(df)] = amps
            except IndexError:
                pass
        df.to_csv(path2 + param + '/' + file + param + '.csv', sep='\t')
    print('Done')

010719ctrol1_analysis_result_ctrol1a0.json
010719ctrol1_analysis_result_ctrol1b0.json
010719ctrol8_analysis_result_ctrol8a0.json
010719ctrol8_analysis_result_ctrol8b0.json
010719Dan2_analysis_result_dan2a000.json
010719Dan2_analysis_result_dan2b000.json
010719Dan6_analysis_result_dan6a000.json
010719Dan6_analysis_result_dan6b000.json
010719VK3_analysis_result_vk3a000.json
010719VK3_analysis_result_vk3b002.json
010719VK7_analysis_result_vk7a000.json
010719VK7_analysis_result_vk7b000.json
010819c1_analysis_result_c1a000.json
010819c1_analysis_result_c1b000.json
010819c2_analysis_result_c2a000.json
010819c2_analysis_result_c2b000.json
010819c3_analysis_result_c3a000.json
010819c3_analysis_result_c3b001.json
010819c3_analysis_result_c3c000.json
031019c1_analysis_result_c1b000.json
031019c1_analysis_result_c1e000.json
031019c2_analysis_result_c2b000.json
031019c2_analysis_result_c2e000.json
031019c2_analysis_result_c2f000.json
031019c3_analysis_result_c3b000.json
031019c3_analysis_result_c3

In [13]:
# Guarda en una tabla los datos de la célula entera

path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
path2 = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/'
jsons_list = os.listdir(path)
os.makedirs(path2 + 'whole_cell/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['image']
    pairs = {k: dic[k] for k in list(dic)[1:]}
    keys = list(pairs.keys())
    df = pd.DataFrame()
    for label in range(0,len(pairs)):
        df[keys[label]] = pairs[keys[label]][0:2]
    df = df.T
    df.columns = ['Transient1', 'Transient2']
    df.to_csv(path2 + 'whole_cell/' + file + '.csv', sep='\t')
print('Done')

Done


In [12]:
# Cálculo del TAU67%

#     '''This function finds a tList in sec yList - measurements ySS - the steady state value of y returns amplitude of exponent tau - the time constant'''

def fitExponent(tList,yList,ySS=0):
    bList = [log(max(y-ySS,1e-6)) for y in yList]
    (w,residuals,rank,sing_vals) = lstsq(matrix([[1,t] for t in tList]),matrix(bList).T, rcond=None)
    tau = -1.0/w[1,0]
    amplitude = exp(w[0,0])
    return (amplitude,tau)

path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/'
jsons_list = os.listdir(path)
# os.makedirs(path2 + 'taus_preliminar/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['slices']
    TAU = pd.DataFrame()
    for peak in range  (0, len(dic)):
        amp = ((np.array(dic[peak]['max_peaks_intensities'][0:2]) - np.array(dic[peak] ['min_peaks_intensities'][1:3]))/np.exp(1))+np.array(dic[peak] ['min_peaks_intensities'][1:3])
        tau_peaks = []
        for i in range(0,len(amp)):
            array = dic[peak]['intensities']
            value = amp[i]
            trans_list = dic[peak]['intensities'][dic[peak]['max_peaks_positions'][i]:dic[peak]['min_peaks_positions'][i+1]]
            x = np.array(range(0,len(trans_list)))
            try:
                (amplitudeEst,tauEst) = fitExponent(x,trans_list)
                yEst = amplitudeEst*(exp(-x/tauEst))
                tau_peaks.append(-tauEst*log(value/amplitudeEst))
            except:
                tau_peaks.append('nan')
        TAU[peak] = tau_peaks
    taus = pd.DataFrame(TAU.T.values, columns = ['transient1','transient2'])
#     taus.to_excel(path2 + 'taus_preliminar/' + file + '.xlsx', engine='xlsxwriter')  

    transient1  transient2
0    69.435949   55.301849
1    74.223715   58.278813
2    73.483776   59.812313
3    73.543068   60.361284
4    73.048347   60.092741
5    72.117529   57.912729
6    75.368390   59.228112
7    75.655697   58.933997
8    76.407866   59.974528
9    76.099426   60.415459
10   73.099147   55.583846
11   74.025600   56.045656
12   74.447745   60.059344
13   66.704838   60.143579
14   69.624828   55.003823
15   69.877475   61.572082
16   69.813534   53.504017
17   69.172187   53.409567
18   74.469695   56.038495
19   71.435625   58.757286
20   70.257780   59.631707
21   73.781767   58.970037
22   71.698704   62.708561
23   76.392944   60.559368
24   74.123233   59.540430
25   73.347312   58.609716
26   77.101048   60.981891
27   70.439378   57.863407
28   71.562504   59.746951
29   72.033609   61.411111
30   73.547987   62.271291
31   75.012472   61.471948
32   77.358397   63.245218
33   78.386343   57.772448
34   71.380960   58.298933
35   73.267279   60.223428
3

<ipython-input-12-8a4deaae9f48>:8: RuntimeWarning: divide by zero encountered in double_scalars
  tau = -1.0/w[1,0]


    transient1  transient2
0    15.615965   18.856211
1    12.825474   22.000743
2    14.929454   23.445489
3    18.415454   22.997626
4    16.412760   22.102915
5    16.198500   22.102055
6    17.837185   26.183906
7    21.378122   22.587745
8    21.058418   24.285642
9    17.927188   24.123960
10   19.631420   26.861404
11   24.183339   26.878041
12   23.577864   22.680804
13   24.026080   15.467270
14   21.964301   17.612980
15   18.684867   21.869295
16   15.405034   20.344254
17   36.737084    8.320613
18   53.273265   12.923038
19   20.182460   14.576493
20   19.535687   21.050681
21   16.421538   18.990916
22   21.493284   14.249702
23   19.581195   13.967143
24   19.882159   23.187775
25   33.490791   25.238968
26   30.242921   23.857691
27   16.801016   24.674789
28        -inf   22.919891
29    2.619577   29.152128
30  528.601782   26.769541
31   12.083814   24.849359
32   18.699364   19.734097
33   11.123463   20.251060
34   12.886453   20.606808
    transient1  transient2
0

<ipython-input-12-8a4deaae9f48>:8: RuntimeWarning: divide by zero encountered in double_scalars
  tau = -1.0/w[1,0]


    transient1  transient2
0    40.464643   38.780449
1    36.558762   42.619479
2    41.735417   44.779328
3    39.943975   47.710530
4    42.607885   46.834267
5    43.561933   45.189322
6    42.443327   42.199367
7    43.890784   38.865012
8    43.014373   38.528020
9    45.692749   44.634915
10   44.686383   42.726058
11   44.833790   45.037821
12   45.721400   46.301536
13   44.065388   43.955820
14   46.687334   44.143447
15   44.832314   47.078352
16   45.703753   46.656642
17   45.504615   47.145586
18   45.652037   44.097799
19   45.159447   42.935005
20   38.617030   39.671291
21   38.569757   40.512571
22   40.454867   40.983735
23   46.069938   45.191171
24   48.142595   46.780241
25   49.303220   47.373792
26   49.475611   48.567167
27   48.123602   48.571112
28   45.336586   45.893896
29   46.267645   45.121026
30   42.443833   41.345226
31   39.924734   42.359650
32   38.425196   42.092257
33   41.867041   44.376119
34   43.869409   43.764297
35   36.967074   38.424795
3

...

<ipython-input-12-8a4deaae9f48>:8: RuntimeWarning: divide by zero encountered in double_scalars
  tau = -1.0/w[1,0]
<ipython-input-12-8a4deaae9f48>:8: RuntimeWarning: divide by zero encountered in double_scalars
  tau = -1.0/w[1,0]


      transient1  transient2
0   1.888357e+01   17.089446
1   1.891710e+01   20.914958
2   1.902028e+01   16.704836
3   1.837648e+01   17.089446
4   1.516440e+01   15.272578
5   1.730369e+01    9.606442
6   1.918717e+01   14.062853
7   1.937692e+01   16.485107
8   2.396454e+01   10.780499
9   1.623467e+01   19.187772
10  1.374286e+01   20.352331
11  1.994248e+01   19.786342
12  1.709294e+01   19.440580
13  2.944658e+01   20.184043
14  3.795033e+14   21.733011
15  1.052748e+01   19.304868
16  1.556200e+01   23.506158
17  1.629902e+01   20.137963
18  1.571698e+01   16.934554
19  1.779344e+01   16.343380
20  1.689556e+01   14.758401
21  1.954088e+01   16.901732
22  2.013025e+01   11.888814
    transient1  transient2
0    47.756514   49.371690
1    53.137478   47.581855
2    51.744515   47.206040
3    49.160119   48.376377
4    47.972524   48.367186
5    50.988753   50.544595
6    48.560577   47.863550
7    50.067857   53.851777
8    56.499269   58.648034
9    65.536968   61.516053
10   71

<ipython-input-12-8a4deaae9f48>:8: RuntimeWarning: divide by zero encountered in double_scalars
  tau = -1.0/w[1,0]


    transient1  transient2
0    18.654160   18.172279
1    20.566401   22.578380
2    19.327384   22.403008
3    17.667653   18.671573
4    13.030359   17.612683
5    18.031371   17.943846
6    15.676416   16.841242
7    12.770916   15.894905
8    14.337320   18.243242
9    21.835731   17.444698
10   23.719991   19.275847
11   20.109147   22.255005
12   24.268915   19.575036
13   19.792742   15.810457
14   18.592896   15.223018
15   14.973512   16.333761
16   14.496241   14.970606
17   13.605937   25.337214
18   16.978132   19.137930
19   10.011879   12.893438
20   20.019248   24.398577
21   14.635344   22.454032
22   16.309698   13.643005
23   13.905281   14.818520
24   17.367582   17.750756
25   14.057359   14.609098
26   16.439903   13.931945
27   20.558009   13.663277
    transient1  transient2
0    57.329404   43.623553
1    64.523008   46.548245
2    65.525642   46.691075
3    70.293537   50.393596
4    71.636531   56.727442
5    73.376543   59.379549
6    71.017103   56.112031
7

<ipython-input-12-8a4deaae9f48>:8: RuntimeWarning: divide by zero encountered in double_scalars
  tau = -1.0/w[1,0]


In [20]:
# Cálculo del TAU67% whole cell

#     '''This function finds a tList in sec yList - measurements ySS - the steady state value of y returns amplitude of exponent tau - the time constant'''

def fitExponent(tList,yList,ySS=0):
    bList = [log(max(y-ySS,1e-6)) for y in yList]
    (w,residuals,rank,sing_vals) = lstsq(matrix([[1,t] for t in tList]),matrix(bList).T, rcond=None)
    tau = -1.0/w[1,0]
    amplitude = exp(w[0,0])
    return (amplitude,tau)

path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
path2 = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/'
jsons_list = os.listdir(path)
TAU = pd.DataFrame()
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['image']
    amp = ((np.array(dic['max_peaks_intensities'][0:2]) - np.array(dic['min_peaks_intensities'][1:3]))/np.exp(1))+np.array(dic['min_peaks_intensities'][1:3])
    tau_peaks = []
    for i in range(0,len(amp)):
        array = dic['intensities']
        value = amp[i]
        trans_list = dic['intensities'][dic['max_peaks_positions'][i]:dic['min_peaks_positions'][i+1]]
        x = np.array(range(0,len(trans_list)))
        try:
            (amplitudeEst,tauEst) = fitExponent(x,trans_list)
            yEst = amplitudeEst*(exp(-x/tauEst))
            tau_peaks.append(-tauEst*log(value/amplitudeEst))
        except:
            tau_peaks.append('nan')
    TAU[file] = tau_peaks
taus = pd.DataFrame(TAU.T.values, columns = ['transient1','transient2'])
taus['file'] = TAU.T.index
taus.to_excel(path2 + 'wc_tau_preliminar' + '.xlsx', engine='xlsxwriter')  

[46, 46, 46, 46, 46, 45, 45, 45, 45, 45, 45, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 43, 42, 42, 42, 42]
42.83939720585721
19.462361398570884
[45, 45, 45, 45, 45, 45, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 41, 41, 41, 41]
41.83939720585721
18.896675474782104


In [9]:
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/taus_preliminar/'
experiments = os.listdir(path)
taus_exp = pd.DataFrame()
for i in range(0,len (experiments)):
    experiment = experiments[i]
    tau_file = pd.read_excel(path + experiment)
    del tau_file['Unnamed: 0']
    taus_exp[experiment] = list(tau_file.mean())
local_taus_mean = taus_exp.T
local_taus_mean.columns = ['transient1','transient2']
local_taus_mean['file'] = local_taus_mean.index
local_taus_mean = local_taus_mean.reset_index(drop=True)

In [10]:
tau_file = pd.read_excel('C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/wc_tau_preliminar.xlsx')
del tau_file['Unnamed: 0']

In [11]:
tabulado = pd.read_csv('C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/tabulado')
del tabulado['Unnamed: 0']

In [12]:
tau_file.head()

,transient1,transient2,file
0,56.642803,50.441796,010719ctrol1_analysis_result_ctrol1a0.json
1,21.848068,22.281768,010719ctrol1_analysis_result_ctrol1b0.json
2,17.606736,17.085831,010719ctrol8_analysis_result_ctrol8a0.json
3,108.615258,95.514131,010719ctrol8_analysis_result_ctrol8b0.json
4,65.503319,55.030121,010719Dan2_analysis_result_dan2a000.json


In [55]:
# Filtrado por tratamiento y frecuencia
tratamiento = ['C','VK','DAN']
frecuencia = [1,5]
def tau_filter(tratamiento,frecuencia,df):
    tabulado_filter = tabulado[(tabulado['tratamiento']==tratamiento) & (tabulado['frecuencia']== frecuencia)].reset_index(drop=True)
    tau = pd.DataFrame(columns = ['transient1','transient2','file'])
    for j in range(0,len (tabulado_filter)):
        celula = tabulado_filter['célula'][j]
        foto = tabulado_filter['foto'][j][:-4]
        for i in range(0,len (df)):
            if (celula in df['file'][i]) & (foto in df['file'][i]):
                tau.loc[j] = df.loc[i]
    return tau

In [56]:
def outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    idx = ~((column < (Q1 - 1.5 * IQR)) | (column > (Q3 + 1.5 * IQR)))
    return idx

In [60]:
tau_control1 = tau_filter('C',1,local_taus_mean)
tau_control5 = tau_filter('C',5,local_taus_mean)

In [65]:
tau_control1 = tau_control1[(outliers(tau_control1['transient1']) == True) & (outliers(tau_control1['transient2']) == True)]
tau_control1.dropna()
tau_control1 = tau_control1[tau_control1['transient1'] > 5]
tau_control1.describe()

,transient1,transient2
count,24.000000,24.000000
mean,58.383145,45.822322
std,24.698558,16.465671
min,12.088402,13.433615
25%,42.051213,37.437283
50%,63.791972,48.486573
75%,73.607024,54.620233
max,97.194771,78.454978


In [67]:
tau_control1.set_index('file')

,transient1,transient2
file,,
010819c3_analysis_result_c3a000.json.xlsx,87.436491,49.482156
010819c2_analysis_result_c2a000.json.xlsx,52.432988,51.797645
031019c4_analysis_result_c4b000.json.xlsx,87.819095,73.915790
031019c1_analysis_result_c1b000.json.xlsx,85.659596,78.454978
050320c7_analysis_result_7a001.json.xlsx,75.207489,49.324063
050320c2_analysis_result_2a000.json.xlsx,88.210490,42.833788
050320c8_analysis_result_8b000.json.xlsx,73.073536,59.061467
050320c4_analysis_result_4a001.json.xlsx,65.579057,56.474084
050320c3_analysis_result_3a001.json.xlsx,71.346090,32.131206


In [32]:
import scipy.stats

scipy.stats.ttest_ind(tau_control1['transient1'],tau_control5['transient1'])

Ttest_indResult(statistic=8.042838302308018, pvalue=2.1986838056700396e-10)